In [1]:
import yfinance as yf
import numpy as np

In [3]:
tickers = ["AMZN", "MSFT"]
ohlcv_data = {}

In [4]:
for ticker in tickers:
  temp = yf.download(ticker, period = '1mo', interval = '5m')
  temp.dropna(how = "any", inplace = True)
  ohlcv_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [5]:
def ATR(DF, n=14):
  df = DF.copy()
  df["H-L"] = df["High"] - df["Low"]
  df["H-PC"] = df["High"] - df["Adj Close"].shift(1)
  df["L-PC"] = df["Low"] - df["Adj Close"].shift(1)
  df["TR"] = df[["H-L", "H-PC", "L-PC"]].max(axis = 1, skipna = False)
  df["ATR"] = df["TR"].ewm(com = n, min_periods = n).mean()
  return df["ATR"]

In [14]:
def ADX(DF, n=20):
  df = DF.copy()
  df["ATR"] = ATR(df, n)
  df["upmove"] = df["High"] - df["High"].shift(1)
  df["downmove"] = df["Low"].shift(1) - df["Low"]
  # Use np.where with the conditions to create the '+dm' and '-dm' columns
  df["+dm"] = np.where((df["upmove"] > df["downmove"]) & (df["upmove"] > 0), df["upmove"], 0)
  df["-dm"] = np.where((df["downmove"] > df["upmove"]) & (df["downmove"] > 0), df["downmove"], 0)
  df["+di"] = 100 * (df["+dm"]/df["ATR"]).ewm(span = n, min_periods = n).mean()
  df["-di"] = 100 * (df["-dm"]/df["ATR"]).ewm(span = n, min_periods = n).mean()
  df["ADX"] = 100 * abs((df["+di"] - df["-di"])/(df["+di"] + df["-di"])).ewm(span = n, min_periods = n).mean()
  return df["ADX"]


In [15]:
for ticker in ohlcv_data:
  ohlcv_data[ticker]["ADX"] = ADX(ohlcv_data[ticker], 20)

In [16]:
print(ohlcv_data)

{'AMZN':                                  Open        High         Low       Close  \
Datetime                                                                    
2024-07-01 09:30:00-04:00  193.479996  194.449997  192.820007  193.199997   
2024-07-01 09:35:00-04:00  193.240005  194.300003  193.240005  194.054993   
2024-07-01 09:40:00-04:00  194.039993  194.039993  193.460007  193.729996   
2024-07-01 09:45:00-04:00  193.880005  195.024994  193.710007  194.459900   
2024-07-01 09:50:00-04:00  194.399994  194.672302  193.750000  193.949997   
...                               ...         ...         ...         ...   
2024-07-31 15:35:00-04:00  186.914993  187.304993  186.710007  186.940002   
2024-07-31 15:40:00-04:00  186.952393  187.570007  186.919998  187.460007   
2024-07-31 15:45:00-04:00  187.440002  187.570007  186.690002  186.880005   
2024-07-31 15:50:00-04:00  186.869995  187.339996  186.380005  187.250000   
2024-07-31 15:55:00-04:00  187.259995  187.460007  186.847397  186.